In [ ]:
from PIL import Image
import pytesseract


# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# Path to your image
image = Image.open("../train_32x32_gray_contrast/img_000004.jpg")

# Extract text
text = pytesseract.image_to_string(image)

print(text)

In [ ]:
from PIL import Image, ImageOps, ImageFilter
import pytesseract

# Optional: Set path to tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load image
image = Image.open("../train_32x32_gray_contrast/img_000004.jpg")

# Preprocessing
# Convert to grayscale
gray = image.convert("L")

# Apply a binary threshold (simple)
threshold = 128
bw = gray.point(lambda x: 255 if x > threshold else 0, mode='1')

# Optional: resize to help OCR
bw = bw.resize((bw.width * 2, bw.height * 2), Image.LANCZOS)
bw.show()
# OCR
text = pytesseract.image_to_string(bw)

print("Detected text:")
print(text)

Detected text:



In [15]:
import os
from PIL import Image
import pytesseract

# Optional: Set path to Tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Directory containing images
folder_path = "./train/"
custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789'
# Loop through all files
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)

        # Preprocess image
        gray = image.convert("L")
        threshold = 128
        bw = gray.point(lambda x: 255 if x > threshold else 0, mode='1')
        bw = bw.resize((bw.width * 2, bw.height * 2), Image.LANCZOS)

        # OCR
        text = pytesseract.image_to_string(bw, config=custom_config)

        # Only print if any text is found
        if text.strip():
            print(f"\n📄 File: {filename}")
            print("Detected text:")
            print(text.strip())


📄 File: img_000008.jpg
Detected text:
0

📄 File: img_000009.jpg
Detected text:
3

📄 File: img_000049.jpg
Detected text:
4

📄 File: img_000065.jpg
Detected text:
6

📄 File: img_000120.jpg
Detected text:
1

📄 File: img_000125.jpg
Detected text:
6

📄 File: img_000182.jpg
Detected text:
169

📄 File: img_000200.jpg
Detected text:
1

📄 File: img_000201.jpg
Detected text:
7

📄 File: img_000213.jpg
Detected text:
4

📄 File: img_000263.jpg
Detected text:
4

📄 File: img_000297.jpg
Detected text:
7

📄 File: img_000313.jpg
Detected text:
3


KeyboardInterrupt: 

In [159]:
from PIL import Image, ImageEnhance
import pytesseract
import re
import numpy as np

# Load image
image_path = './train/img_000101.jpg'
image = Image.open(image_path)

# Get image dimensions
width, height = image.size

# Define crop box (center square or rectangle)
crop_width = int(width * 0.5)   # crop 50% width around center
crop_height = int(height * 0.4) # crop 40% height around center

left = (width - crop_width) // 2
top = (height - crop_height) // 2
right = left + crop_width
bottom = top + crop_height

# Crop first
cropped = image.crop((left, top, right, bottom))

# Convert cropped image to grayscale
gray = cropped.convert('L')

# Convert to NumPy array for brightness analysis
gray_np = np.array(gray)
mean_brightness = gray_np.mean()
print(f"Mean brightness: {mean_brightness:.2f}")

# Dynamically choose brightness factor
if mean_brightness < 20:
    factor = 15
if mean_brightness < 60:
    factor = 5
elif mean_brightness < 100:
    factor = 1.5
elif mean_brightness < 150:
    factor = 1.2
else:
    factor = 1.0
print(f"Brightness enhance factor: {factor}")

# Brighten grayscale image
enhancer = ImageEnhance.Brightness(gray)
brightened = enhancer.enhance(factor)

# ✨ Enhance contrast
contrast_enhancer = ImageEnhance.Contrast(brightened)
brightened = contrast_enhancer.enhance(5)  # Try 1.2 - 2.0 if needed

# Apply binary thresholding
threshold = 128
bw = brightened.point(lambda x: 255 if x > threshold else 0, mode='1')

# Optional: show preprocessed image
bw.show()

# Resize to help OCR
bw = bw.resize((bw.width * 2, bw.height * 2), Image.LANCZOS)

# OCR config for digits only
custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789'

# Run OCR
raw_text = pytesseract.image_to_string(bw, config=custom_config)

# Extract digits
numbers_only = re.findall(r'\d+', raw_text)

if numbers_only:
    print("✅ Detected numbers:", ' '.join(numbers_only))
else:
    print("❌ No digits detected.")

Mean brightness: 169.07
Brightness enhance factor: 1.0
❌ No digits detected.


# Dataframe conversion

In [162]:
import os
import re
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
import pytesseract
from tqdm import tqdm

# Directory with images
folder_path = './train/'

# Store results
data = []

# Loop through image files
for filename in tqdm(os.listdir(folder_path)):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")):
        image_path = os.path.join(folder_path, filename)

        try:
            # Load image
            image = Image.open(image_path)

            # Crop center
            width, height = image.size
            crop_width = int(width * 0.5)
            crop_height = int(height * 0.4)
            left = (width - crop_width) // 2
            top = (height - crop_height) // 2
            right = left + crop_width
            bottom = top + crop_height
            cropped = image.crop((left, top, right, bottom))

            # Grayscale
            gray = cropped.convert('L')
            gray_np = np.array(gray)
            mean_brightness = gray_np.mean()

            # Dynamic brightness factor
            if mean_brightness < 20:
                factor = 15
            elif mean_brightness < 60:
                factor = 5
            elif mean_brightness < 100:
                factor = 1.5
            elif mean_brightness < 150:
                factor = 1.2
            else:
                factor = 1.0

            # Brighten
            enhancer = ImageEnhance.Brightness(gray)
            brightened = enhancer.enhance(factor)

            # Contrast boost
            contrast_enhancer = ImageEnhance.Contrast(brightened)
            brightened = contrast_enhancer.enhance(5)

            # Threshold
            threshold = 128
            bw = brightened.point(lambda x: 255 if x > threshold else 0, mode='1')

            # Resize
            bw = bw.resize((bw.width * 2, bw.height * 2), Image.LANCZOS)

            # OCR
            custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=0123456789'
            raw_text = pytesseract.image_to_string(bw, config=custom_config)

            # Extract digits
            digits_found = set(re.findall(r'\d', raw_text))

            # Record presence of each digit
            row = {'image_path': image_path}
            for digit in '0123456789':
                row[digit] = 1 if digit in digits_found else 0
            data.append(row)

        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")

# Create DataFrame
df = pd.DataFrame(data)

# Save or display
print(df.head())
# df.to_csv("ocr_digit_detection.csv", index=False)

100%|██████████| 5490/5490 [16:25<00:00,  5.57it/s]


               image_path  0  1  2  3  4  5  6  7  8  9
0  ./train/img_000001.jpg  0  0  0  0  0  0  0  0  0  0
1  ./train/img_000002.jpg  0  0  0  0  0  0  0  0  0  0
2  ./train/img_000003.jpg  0  0  0  0  0  0  0  0  0  0
3  ./train/img_000004.jpg  1  0  0  0  0  0  0  1  0  0
4  ./train/img_000005.jpg  0  0  0  0  0  0  0  0  0  0


In [163]:
# Get digit columns (exclude 'image_path')
digit_columns = [str(d) for d in range(10)]

# Filter rows where at least one digit is present
filtered_df = df.loc[df[digit_columns].any(axis=1)].reset_index(drop=True)

# Display or save
filtered_df.head(20)
# filtered_df.to_csv("filtered_ocr_digits.csv", index=False)

,image_path,0,1,2,3,4,5,6,7,8,9
0,./train/img_000004.jpg,1,0,0,0,0,0,0,1,0,0
1,./train/img_000008.jpg,0,0,0,0,1,0,0,0,0,0
2,./train/img_000024.jpg,1,1,0,0,0,0,0,0,0,0
3,./train/img_000037.jpg,1,1,0,0,0,0,0,0,0,0
4,./train/img_000038.jpg,0,0,0,0,0,0,0,1,0,0
5,./train/img_000051.jpg,0,1,0,0,1,0,0,0,0,0
6,./train/img_000059.jpg,0,0,0,0,0,0,0,1,0,0
7,./train/img_000062.jpg,1,0,0,0,0,0,1,0,0,0
8,./train/img_000067.jpg,1,0,0,0,0,1,0,0,0,0
9,./train/img_000068.jpg,1,0,0,1,0,0,0,0,0,0


In [164]:
df["image_path"] = df["image_path"].str.replace("./train/", "", regex=False)
df

,image_path,0,1,2,3,4,5,6,7,8,9
0,img_000001.jpg,0,0,0,0,0,0,0,0,0,0
1,img_000002.jpg,0,0,0,0,0,0,0,0,0,0
2,img_000003.jpg,0,0,0,0,0,0,0,0,0,0
3,img_000004.jpg,1,0,0,0,0,0,0,1,0,0
4,img_000005.jpg,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5483,img_005484.jpg,0,0,0,0,0,0,0,0,0,0
5484,img_005485.jpg,0,0,0,0,0,0,0,0,0,0
5485,img_005486.jpg,1,0,0,0,0,1,0,0,0,0
5486,img_005487.jpg,1,0,0,0,1,0,1,0,0,0


In [165]:
df.to_csv("ocr_digit_detection.csv", index=False)